<h2 style='color:yellow' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")  
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [5]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [9]:
dummies.replace({True: 1, False: 0}, inplace=True)

In [10]:
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [11]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [12]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [13]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [14]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [15]:
inputs.Age.mean()

29.69911764705882

In [18]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head() 

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3,random_state=10)
print(len(X_train),len(X_test))

623 268


In [26]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [27]:
model.fit(X_train,y_train)

GaussianNB()

In [28]:
model.score(X_test,y_test)

0.8059701492537313

In [29]:
X_test[0:10]

,Pclass,Age,Fare,female
590,3,35.0,7.1250,0
131,3,20.0,7.0500,0
628,3,26.0,7.8958,0
195,1,58.0,146.5208,1
230,1,35.0,83.4750,1
646,3,19.0,7.8958,0
75,3,25.0,7.6500,0
586,2,47.0,15.0000,0
569,3,32.0,7.8542,0
287,3,22.0,7.8958,0


In [30]:
y_test[0:10]

590    0
131    0
628    0
195    1
230    1
646    0
75     0
586    0
569    1
287    0
Name: Survived, dtype: int64

In [31]:
model.predict(X_test[0:10])

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [32]:
model.predict_proba(X_test[:10])

array([[9.58081706e-01, 4.19182944e-02],
       [9.54611200e-01, 4.53888002e-02],
       [9.57799697e-01, 4.22003028e-02],
       [2.83651155e-04, 9.99716349e-01],
       [2.51214754e-02, 9.74878525e-01],
       [9.54115147e-01, 4.58848527e-02],
       [9.57401477e-01, 4.25985230e-02],
       [9.07071629e-01, 9.29283708e-02],
       [9.58609767e-01, 4.13902330e-02],
       [9.56084206e-01, 4.39157941e-02]])

**Calculate the score using cross validation**

In [43]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.744     , 0.76      , 0.736     , 0.73387097, 0.80645161])

In [40]:
avge=cross_val_score(GaussianNB(),X_train, y_train, cv=5)

In [41]:
import numpy as np
print(np.mean(avge))

0.7560645161290322
